In [1]:
from ib_insync import *
import pandas as pd
import random
from datetime import date, time

In [7]:
# get the list of tickers
symbol_list = list(pd.read_csv("data/stocks_daily.csv")["isin"].unique())
symbol_list

['US19260Q1076',
 'US83304A1060',
 'DE000A3ENQ51',
 'DE0005545503',
 'US09075V1026',
 'US92826C8394',
 'US00287Y1091',
 'LU1778762911',
 'US58933Y1055',
 'US0090661010',
 'US12468P1049',
 'US40434L1052',
 'US02079K3059',
 'US02079K1079',
 'CA06849F1080',
 'DE000A0Z23Q5',
 'DE000A1EWWW0',
 'US0258161092',
 'DE0005313704',
 'DE000A2LQ884',
 'DE000A0WMPJ6',
 'DE0008404005',
 'US0079031078',
 'US0311621009',
 'DE000AUM0V10',
 'US0231351067',
 'US0378331005',
 'LU1673108939',
 'DE000BASF111',
 'DE000BAY0017',
 'DE0005158703',
 'US0970231058',
 'DE0005200000',
 'LU1704650164',
 'DE0005190003',
 'DE0005190037',
 'DE000A1DAHH0',
 'DE000A1PHFF7',
 'US1101221083',
 'US0846707026',
 'US1491231015',
 'DE000CBK1001',
 'US1912161007',
 'US1921085049',
 'US1667641005',
 'US17275R1023',
 'US46625H1005',
 'DE0005419105',
 'DE0005439004',
 'US1941621039',
 'DE0005403901',
 'US1713401024',
 'DE0005810055',
 'DE0005140008',
 'US2441991054',
 'DE0006305006',
 'DE000A2E4K43',
 'DE0005552004',
 'DE000A2GS5D8

In [13]:


# 1. Setup
util.startLoop() 
ib = IB()

# 2. Connect
my_client_id = random.randint(1, 9999)
try:
    ib.connect('127.0.0.1', 7497, clientId=my_client_id)
    print("✅ Connected!")
except Exception as e:
    print(f"❌ Error: {e}")

# 3. Market Data Type (3 = Delayed)
ib.reqMarketDataType(3)


# scratch the data
price_data_ls = []
for idx, isin in enumerate(symbol_list):

    # resolve ISIN
    contract = Contract(
        secType='STK',
        secIdType='ISIN',
        secId=isin,
        exchange='SMART'
    )
    details = ib.reqContractDetails(contract)
    contract_ls = [contract for contract in details if contract.contract.primaryExchange == "IBIS"]
    if len(contract_ls) == 0:
        continue
    contract = contract_ls[0].contract
    symbol_local = contract.localSymbol
    symbol = contract.symbol
    conId = contract.conId

    # get the ticker data
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='30 D',
        barSizeSetting='5 mins',
        whatToShow='TRADES',
        useRTH=True
    )

    # pandas Processing
    df = util.df(bars)
    if df is not None and not df.empty:
        df['date'] = pd.to_datetime(df['date'])
        df = df.rename({"date": "datetime"}, axis=1)
        df["date"] = df["datetime"].dt.date
        df["time"] = df["datetime"].dt.time
        df["hour"] = df["datetime"].dt.hour
        df["symbol_local"] = symbol_local
        df["symbol"] = symbol
        df["isin"] = isin
        df["contract"] = conId
        price_data_ls.append(df)
        print(f"SUCCESS - {round(100*idx/len(symbol_list), 2)}% - {symbol}")
    else:
        print("⚠️ No data received. Try changing 'whatToShow' to 'MIDPOINT' or check market hours.")

# disconnect
ib.disconnect()

# put data together
price_data_df = pd.concat(price_data_ls)
price_data_df.to_csv("data/stocks_5min.csv", index=False)
price_data_df

✅ Connected!
SUCCESS - 0.0% - 1QZ
SUCCESS - 0.48% - 1SI
SUCCESS - 0.96% - 1SXP
SUCCESS - 1.44% - 1U1
SUCCESS - 1.92% - 22UA
SUCCESS - 2.4% - 3V64
SUCCESS - 2.88% - 4AB
SUCCESS - 3.37% - 639
SUCCESS - 3.85% - 6MK
SUCCESS - 4.33% - 6Z1
SUCCESS - 4.81% - 724
SUCCESS - 5.29% - 7HP
SUCCESS - 5.77% - ABEA
SUCCESS - 6.25% - ABEC
SUCCESS - 6.73% - ABR
SUCCESS - 7.21% - ADN1
SUCCESS - 7.69% - ADS
SUCCESS - 8.17% - AEC1
SUCCESS - 8.65% - AFX
SUCCESS - 9.13% - AG1
SUCCESS - 9.62% - AIXA
SUCCESS - 10.1% - ALV
SUCCESS - 10.58% - AMD
SUCCESS - 11.06% - AMG1
SUCCESS - 11.54% - AMV0
SUCCESS - 12.02% - AMZ
SUCCESS - 12.5% - APC
SUCCESS - 12.98% - AT11
SUCCESS - 13.46% - BAS
SUCCESS - 13.94% - BAYN
SUCCESS - 14.42% - BC8
SUCCESS - 14.9% - BCO
SUCCESS - 15.38% - BEI
SUCCESS - 15.87% - BFSA
SUCCESS - 16.35% - BMW
SUCCESS - 16.83% - BMW3
SUCCESS - 17.31% - BNR
SUCCESS - 17.79% - BOSS
SUCCESS - 18.27% - BRM
SUCCESS - 18.75% - BRYN
SUCCESS - 19.23% - CAT1
SUCCESS - 19.71% - CBK
SUCCESS - 20.19% - CCC3
SUCCES

,datetime,open,high,low,close,volume,average,barCount,date,time,hour,symbol_local,symbol,isin,contract
0,2025-12-16 09:00:00+01:00,210.840,211.66,210.840,211.40,276.0,211.4160,10,2025-12-16,09:00:00,9,1QZ,1QZ,US19260Q1076,482628319
1,2025-12-16 09:05:00+01:00,211.600,211.90,211.400,211.90,920.0,211.6920,32,2025-12-16,09:05:00,9,1QZ,1QZ,US19260Q1076,482628319
2,2025-12-16 09:10:00+01:00,211.840,212.20,211.500,211.50,1264.0,211.8900,46,2025-12-16,09:10:00,9,1QZ,1QZ,US19260Q1076,482628319
3,2025-12-16 09:15:00+01:00,211.500,211.80,211.500,211.80,27.0,211.6560,2,2025-12-16,09:15:00,9,1QZ,1QZ,US19260Q1076,482628319
4,2025-12-16 09:20:00+01:00,211.500,211.80,211.300,211.80,158.0,211.4000,10,2025-12-16,09:20:00,9,1QZ,1QZ,US19260Q1076,482628319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,2026-02-02 16:15:00+01:00,24.290,24.45,24.290,24.43,16901.0,24.3905,96,2026-02-02,16:15:00,16,ZAL,ZAL,DE000ZAL1111,168731675
3007,2026-02-02 16:20:00+01:00,24.430,24.56,24.400,24.56,46975.0,24.4780,122,2026-02-02,16:20:00,16,ZAL,ZAL,DE000ZAL1111,168731675
3008,2026-02-02 16:25:00+01:00,24.570,24.62,24.510,24.62,11240.0,24.5680,68,2026-02-02,16:25:00,16,ZAL,ZAL,DE000ZAL1111,168731675
3009,2026-02-02 16:30:00+01:00,24.640,24.67,24.620,24.62,8468.0,24.6455,57,2026-02-02,16:30:00,16,ZAL,ZAL,DE000ZAL1111,168731675
